In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')

In [ ]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [ ]:
drop_indices = np.random.choice(diabetes.index, 368, replace=False)
diabetes = diabetes.drop(drop_indices)

In [ ]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'], test_size=0.2)

In [ ]:
X_train[X_train == 0] = 0.000001
X_test[X_test == 0] = 0.000001

In [ ]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [ ]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

In [ ]:
"""
with open('results/version4/train_data.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version4/test_data.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

In [2]:
with open('results/version4/train_data.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version4/train_data.json') as json_file:
    y_train = json.load(json_file)[1]
    
with open('results/version4/test_data.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version4/test_data.json') as json_file:
    y_test = json.load(json_file)[1]

## Test

In [3]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=1, 
                           trainingBias=False,
                           epoch_results=True,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs','phase-encoding'})

lrParameter:  0.1
threshold:  0.7
trainingBias:  False
phaseEstrategyOperator:  original

input:  [0.058823529411764705, 0.5025125628140703, 0.5409836065573771, 0.15151515151515152, 0.06619385342789598, 0.3517138599105813, 0.251067463706234, 0.08333333333333333]
weight:  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

input:  [0.47058823529411764, 0.5477386934673367, 0.6229508196721312, 0.3939393939393939, 0.1347517730496454, 0.4157973174366617, 0.23996584116140052, 0.16666666666666666]
weight:  [0.5058823529411764, 0.550251256281407, 0.5540983606557377, 0.5151515151515151, 0.5066193853427896, 0.5351713859910582, 0.5251067463706234, 0.5083333333333333]

input:  [0.17647058823529413, 0.8844221105527639, 0.7049180327868853, 0.2727272727272727, 0.18439716312056736, 0.496274217585693, 0.45943637916310837, 0.5166666666666667]
weight:  [0.5058823529411764, 0.550251256281407, 0.5540983606557377, 0.5151515151515151, 0.5066193853427896, 0.5351713859910582, 0.5251067463706234, 0.5083333333333333]

inp

In [4]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=5,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs', 'phase-encoding'})

AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.3125


In [ ]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

#print('phase neuron_outputs: ', results[1])
#print('HSGS neuron_outputs: ', results[4])
#print('targets: ', results[6])
#print('examples: ', results[7])

## Search

In [3]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [4]:
# search space

initial_weights = [
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 #[0.9091283748867313, 0.13316944575925016],
 #[0.5234125806737658, 0.7504098591020348],
 #[0.6690132408839138, 0.4677528597449807],
 #[0.20484909029779508, 0.49076588909107044],
 #[0.372384689385059, 0.4774011548515884]
]

threshold_space = [0.8, 0.2, 0.5]
lr_space = [0.01, 0.05, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=w,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=40, 
                                               trainingBias=bias,
                                               epoch_results=True,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs','phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=5,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'hsgs', 'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)

                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')


lrParameter:  0.01
threshold:  0.8
trainingBias:  False
phaseEstrategyOperator:  original

erro HSGS 196
weights HSGS [0.8858826929411794, 1.5906030350753777, 1.6030328668852456, 0.8792934892929308, 0.6704737232387749, 1.394038768137109, 0.7926558497011099, 0.831000219999999]
erro phase encoding 124
weights phase encoding [0.12823510411764721, -0.3884924623115577, -0.21918041786885228, 0.23151469151515086, 0.3534272159810881, -0.14856931955290603, 0.2625832621690862, 0.15616662666666628]

erro HSGS 196
weights HSGS [1.271765385882354, 2.681206070150756, 2.7060657337704943, 1.258586978585856, 0.8409474464775497, 2.2880775362742156, 1.0853116994022196, 1.1620004399999957]
erro phase encoding 124
weights phase encoding [-0.2435297917647056, -1.2769849246231157, -0.938360835737705, -0.03697061696969734, 0.2068544319621762, -0.7971386391058126, 0.02516652433817245, -0.18766674666666713]

erro HSGS 196
weights HSGS [1.6576480788235257, 3.771809105226133, 3.8090986006557452, 1.637880467878779

In [ ]:
experiment_data = pd.DataFrame(output)
experiment_data

In [8]:
experiment_data.to_csv('results/experiment_diabetes.csv')